In [14]:
import torch
import numpy as np
import random

# XMLをファイルやテキストから読み込んだり、加工したり、保存したりするためのライブラリ
import xml.etree.ElementTree as ET

import cv2

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import torch.utils.data as data

In [8]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [11]:
def get_datapath_list(root_dir):
    
    root_dir = Path(root_dir)
    img_dir = root_dir / "JPEGImages"
    ann_dir = root_dir / "Annotations"
    
    train_id_name_txt = root_dir / "ImageSets" / "Main" / "train.txt"
    valid_id_name_txt = root_dir / "ImageSets" / "Main" / "val.txt"
    
    train_img_list = []
    train_ann_list = []
    
    for line in open(train_id_name_txt):
        file_id = line.strip()
        train_img_list.append(img_dir / f"{file_id}.jpg")
        train_ann_list.append(ann_dir / f"{file_id}.xml")
    
    valid_img_list = []
    valid_ann_list = []
    
    for line in open(valid_id_name_txt):
        file_id = line.strip()
        valid_img_list.append(img_dir / f"{file_id}.jpg")
        valid_ann_list.append(ann_dir / f"{file_id}.xml")

    return train_img_list, train_ann_list, valid_img_list, valid_ann_list

### ファイル確認

In [12]:
# ファイルパスのリストを作成
root_dir  = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = get_datapath_list(root_dir)

In [36]:
class Xml2ListConverter():
    
    def __init__(self, classes):
        self.classes = classes
        
    def __call__(self, xml_path, widht, height):
        
        """
        
        [[xmin, ymin, xmax, ymax, label]...]
        """
        
        anno_list = []
        
        xml = ET.parse(xml_path).getroot()
        
        # 難しいもの削除
        for obj in xml.iter("object"):
            
            if int(obj.find('difficult').text) == 1:
                continue
                
            bnd_box = []
            
            name = obj.find('name').text.lower().strip()
            bbox = obj.find('bndbox')  
            
            pts = ['xmin', 'ymin', 'xmax', 'ymax']
            
            for pt in pts:
                #VOCの原点移動
                size = int(bbox.find(pt).text) - 1
                size = size/ height if pt.find("y") > -1 else size / width
                
                bnd_box.append(size)
            
            label_idx = self.classes.index(name)
            
            bnd_box.append(label_idx)
            
            anno_list.append(bnd_box)
   
        return np.array(anno_list)

In [40]:
# クラス定義
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

xml_converter = Xml2ListConverter(voc_classes)

ind = 10
image_file_path = val_img_list[ind]
# Pathlibば読めない
img = cv2.imread(str(image_file_path))  
height, width, channels = img.shape  

# アノテーションをリストで表示
xml_converter(val_anno_list[ind], width, height)

array([[0.888     , 0.67466667, 0.998     , 0.89066667, 6.        ]])

## データの前処理と準備 
- Datasetクラス
- Dataloader
- transfomerを作成

In [42]:
class Pipeline():
    def __init__(self, transforms):
        self.transforms = transforms
        
    def __call__(self, img, boxes=None, labels=None):
        for transform in self.transformes:
            # 必ず引数が3つ必要
            # 予測時はboxes, labels共にNoneなので、それを前提に作成数するひつつ用あり
            img, boxes, labels = transforme(img, boxex, labels)

## 変形を定義

In [44]:
class Preprocessor:
    
    def __init__(self, input_size, color_mean):
        self.preprocess - {
            "train": Pipeline([
                
            ]),
            "val": Pipeline([
                
            ])
        }
        
    
    def __call__(self, img, mode, boxes, labels):
        
        return self.preprocess[mode](img, boxes=None, labels=None)